# A Variational Autoencoder for Single Cell Transcriptomics in the CELLxGENE Dataset

## Background

### Dataset

### Variational autoencoders

In [1]:
from autoCell.data_loader import SingleCellDataset
import src
import cellxgene_census
import anndata
import torch
import pandas as pd
import numpy as np

## Data loading and preprocessing

In [4]:
with cellxgene_census.open_soma() as census:
    adata = cellxgene_census.get_anndata(
        census, 
        "Homo sapiens",
        obs_coords=slice(0, 100),
        obs_value_filter="tissue_general == 'lung' and disease in ['normal','lung adenocarcinoma', 'squamous cell lung carcinoma', 'small cell lung carcinoma', 'non-small cell lung carcinoma', 'pleomorphic carcinoma', 'lung large cell carcinoma'] and is_primary_data == True",  # Specific tissue
        # var_value_filter="feature_name in ['GAPDH', 'ACTB']",  # Specific genes
        obs_column_names=["cell_type", "tissue", "disease"]  # Minimal metadata
    )

print(adata)

The "stable" release is currently 2025-01-30. Specify 'census_version="2025-01-30"' in future calls to open_soma() to ensure data consistency.


AnnData object with n_obs × n_vars = 0 × 61888
    obs: 'cell_type', 'tissue', 'disease', 'tissue_general', 'is_primary_data'
    var: 'soma_joinid', 'feature_id', 'feature_name', 'feature_type', 'feature_length', 'nnz', 'n_measured_obs'


In [2]:
dataset = SingleCellDataset(
    file_path="data.h5ad",
    cell_subset=[i for i in range(1000)],
    log_transform=True,
    normalize=True,
    scale_factor=1.0
)

/Users/frederik/Repositories/environments/math_ml/lib/python3.11/site-packages/anndata/_core/storage.py:39: ImplicitModificationWarning: X should not be a np.matrix, use np.ndarray instead.
  warnings.warn(msg, ImplicitModificationWarning)
/Users/frederik/Repositories/Leipzig_big_data_praktikum_gene_expression_autoencoder/autoCell/data_loader.py:138: ImplicitModificationWarning: Modifying `X` on a view results in data being overridden
  self.adata.X = X


Dataset loaded: 1000 cells × 61888 genes


In [4]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)

In [5]:
for idx, sample in enumerate(dataloader):
    print(f"{idx}: ")
    print(sample.sum())
    print(sample.shape)
    print(sample.max())
    print(sample)
    if idx >= 10:
        break

0: 
tensor(1502.2156)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
1: 
tensor(911.1226)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
2: 
tensor(855.3485)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
3: 
tensor(899.4514)
torch.Size([1, 61888])
tensor(1.)
tensor([[0.0000, 0.5384, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
4: 
tensor(818.5052)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
5: 
tensor(865.3433)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
6: 
tensor(1318.5415)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
7: 
tensor(1082.9573)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
8: 
tensor(1569.1642)
torch.Size([1, 61888])
tensor(1.)
tensor([[0.0000, 0.4118, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])
9: 
tensor(898.8785)
torch.Size([1, 61888])
tensor(1.)
tensor([[0., 0., 0.,  ..., 0.

In [7]:
sample1 = dataset[0]
sample2 = dataset[1]

In [ ]:
from torch.nn import functional as F

def elbo_loss_function(recon_x: torch.Tensor, x: torch.Tensor, mu: torch.Tensor, logvar: torch.Tensor, beta: float=1.0):
    """Uses mean reduction as opposed to summing in the original VAE paper

    TODO: evaluate mean vs sum reduction
    """
    if logvar is None:
        logvar = torch.ones_like(mu)
    
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    print(BCE)
    print(KLD)

    return BCE + beta * KLD

In [56]:
elbo_loss_function(sample1, sample2, torch.Tensor([1.0]), torch.Tensor([1.0]))

tensor(39353.0469)
tensor(0.8591)


tensor(39353.9062)

In [51]:
import keras.backend as K
import keras
import tensorflow as tf

def vae_loss(x, x_decoded_mean, in_dim, var_, z_log_var, z_mean):
                xent_loss = in_dim * keras.metrics.binary_crossentropy(x, x_decoded_mean)
                if var_:
                    kl_loss = - 0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
                else:
                    kl_loss = - 0.5 * tf.reduce_sum(1 + 1 - tf.square(z_mean) - tf.exp(1.0), axis=-1)

                print(xent_loss)
                print(kl_loss)
                return tf.reduce_mean(xent_loss + kl_loss)

In [21]:
tensor1 = tf.convert_to_tensor(sample1.numpy())
tensor2 = tf.convert_to_tensor(sample2.numpy())

In [52]:
vae_loss(tensor1, tensor2, len(tensor2), True, tf.convert_to_tensor([1.0]), tf.convert_to_tensor([1.0]))

tf.Tensor(16710.162, shape=(), dtype=float32)
tf.Tensor(0.8591409, shape=(), dtype=float32)


<tf.Tensor: shape=(), dtype=float32, numpy=16711.021484375>

In [19]:
biase = pd.read_csv("biase.txt", sep="\t")

In [39]:
np.log1p([1.0])

array([0.69314718])

In [36]:
biase = biase.T
biase.max()

GSM1377859    11.931624
GSM1377860    11.227484
GSM1377861    10.835056
GSM1377862    11.658470
GSM1377863    11.937874
GSM1377864    12.529428
GSM1377865    11.865965
GSM1377866    11.022527
GSM1377867    12.017080
GSM1377868    11.572478
GSM1377869    11.073064
GSM1377870    11.014299
GSM1377871    13.280945
GSM1377872    11.918465
GSM1377873    11.931277
GSM1377874    11.457063
GSM1377875    13.050027
GSM1377876    11.713717
GSM1377877    11.508275
GSM1377878    12.708573
GSM1377879    11.920182
GSM1377880    12.992025
GSM1377881    12.230150
GSM1377882    14.065475
GSM1377883    17.889355
GSM1377884    11.902398
GSM1377885    11.494756
GSM1377886    11.253091
GSM1377887    12.624400
GSM1377888    12.351530
GSM1377889    12.144429
GSM1377890    11.421407
GSM1377891    11.425688
GSM1377892    12.090301
GSM1377893    11.810989
GSM1377894    11.534245
GSM1377895    11.317317
GSM1377896    11.864407
GSM1377897    12.053189
GSM1377898    12.676303
GSM1377899    12.355995
GSM1377900    12

## Defining the variational autoencoder

## Training the variational autoencoder

## Evaluation of the latent space

## Discussion